# Baidu Emotion.

### Let's start exploring the dataset

In [1]:
#Adjust the working folder
import sys
import os
#print(globals())
file_folder = globals()['_dh'][0]
wk_dir = os.path.dirname(file_folder)
os.chdir(wk_dir)

import pandas as pd

from code_felix.core.config import *
from code_felix.core.feature import *
from file_cache.utils.util_log import *

import matplotlib.pyplot as plt


plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

2019-01-02 20:14:52,472 util_log.py[61] DEBUG Start the program at:LALI2-M-G0MD, 127.0.0.1, with:Load module


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#Loading the dataset
dataset = pd.read_csv(train_file, encoding='gb18030', delimiter='\t', header=None)

In [3]:
# Prin some samples
dataset.head(3)

,0,1,2,3
0,1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2
1,2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1
2,3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0


## Preparing data for model training
### Tokenization
Since the data is already tokenized and lowercased, we just need to split the words


In [4]:
input_sentences = [list(jieba.cut(str(text), cut_all=False))  for text in dataset.iloc[:, 2].values.tolist()]
labels = dataset.iloc[:, 3].values.tolist()

Building prefix dict from the default dictionary ...
2019-01-02 20:14:52,758 __init__.py[111] DEBUG Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d2/vq91lnt11m13m84s18dzdm8r0000gn/T/jieba.cache
2019-01-02 20:14:52,766 __init__.py[131] DEBUG Loading model from cache /var/folders/d2/vq91lnt11m13m84s18dzdm8r0000gn/T/jieba.cache
Loading model cost 0.715 seconds.
2019-01-02 20:14:53,479 __init__.py[163] DEBUG Loading model cost 0.715 seconds.
Prefix dict has been built succesfully.
2019-01-02 20:14:53,483 __init__.py[164] DEBUG Prefix dict has been built succesfully.


In [5]:
labels[:10]

[2, 1, 0, 0, 1, 2, 2, 2, 2, 2]

### Creating Vocabulary (word index)

In [14]:
word_id_vec =  get_word_id_vec('02')
word2id = dict( word_id_vec.apply(lambda row: (row['word'], row['id']), axis=1).values )
logger.debug(f'Word length:{len(word2id)}')


embedding_weights = word_id_vec.iloc[:, -vector_size:].fillna(0).values
#embedding_weights[10]

2019-01-02 20:16:43,159 util_log.py[41] INFO get_word_id_vec begin with(1 paras) :['02'], []
2019-01-02 20:16:43,175 cache.py[29] DEBUG try to read cache from file:./cache/get_word_id_vec=02=.h5, (h5, key:['/df_0'])
2019-01-02 20:16:43,221 util_log.py[49] INFO get_word_id_vec cost    0.06 sec:(1 paras)(['02'], []), return:DataFrame, end 
2019-01-02 20:16:47,027 <ipython-input-14-102863bb139b>[3] DEBUG Word length:42014


In [7]:
# Initialize word2id and label2id dictionaries that will be used to encode words and labels

label2id = dict()

max_words = 0 # maximum number of words in a sentence

# Construction of word2id dict
for sentence in input_sentences:
#     for word in sentence:
#         # Add words to word2id dict if not exist
#         if word not in word2id:
#             word2id[word] = len(word2id)
#     # If length of the sentence is greater than max_words, update max_words
#     sentence = list(sentence)
#     logger.debug(f'{len(sentence)} : {sentence}')
    if len(sentence) > max_words:
        max_words = len(sentence)
        logger.debug(f'max_words={max_words}')
    
# Construction of label2id and id2label dicts
label2id = {l: i for i, l in enumerate(set(labels))}
id2label = {v: k for k, v in label2id.items()}
id2label

2019-01-02 20:14:58,137 <ipython-input-7-ae333458653d>[18] DEBUG max_words=38
2019-01-02 20:14:58,141 <ipython-input-7-ae333458653d>[18] DEBUG max_words=73
2019-01-02 20:14:58,143 <ipython-input-7-ae333458653d>[18] DEBUG max_words=77
2019-01-02 20:14:58,146 <ipython-input-7-ae333458653d>[18] DEBUG max_words=88
2019-01-02 20:14:58,148 <ipython-input-7-ae333458653d>[18] DEBUG max_words=93
2019-01-02 20:14:58,150 <ipython-input-7-ae333458653d>[18] DEBUG max_words=110
2019-01-02 20:14:58,152 <ipython-input-7-ae333458653d>[18] DEBUG max_words=151
2019-01-02 20:14:58,154 <ipython-input-7-ae333458653d>[18] DEBUG max_words=171
2019-01-02 20:14:58,156 <ipython-input-7-ae333458653d>[18] DEBUG max_words=177
2019-01-02 20:14:58,158 <ipython-input-7-ae333458653d>[18] DEBUG max_words=196


{0: 0, 1: 1, 2: 2}

### Encoding samples with corresponing integer values

In [8]:
import keras

# Encode input words and labels
X = [[word2id[word] for word in sentence] for sentence in input_sentences]
Y = [label2id[label] for label in labels]

# Apply Padding to X
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, max_words)

# Convert Y to numpy array
Y = keras.utils.to_categorical(Y, num_classes=len(label2id))

# Print shapes
print("Shape of X: {}".format(X.shape))
print("Shape of Y: {}".format(Y.shape))


/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/lali2/dev/python/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Shape of X: (2000, 196)
Shape of Y: (2000, 3)


## Build LSTM model with attention 

In [9]:
keras.layers.Embedding?

In [21]:
embedding_dim = 100 # The dimension of word embeddings

# Define input tensor
sequence_input = keras.Input(shape=(max_words,), dtype='int32')

# Word embedding layer
embedded_inputs =keras.layers.Embedding(len(word2id) ,
                                        vector_size ,
                                        input_length=max_words ,
                                        weights = [embedding_weights] ,
                                       )(sequence_input)

# Apply dropout to prevent overfitting
embedded_inputs = keras.layers.Dropout(0.2)(embedded_inputs)

# Apply Bidirectional LSTM over embedded inputs
lstm_outs = keras.layers.wrappers.Bidirectional(
    keras.layers.LSTM(embedding_dim, return_sequences=True)
)(embedded_inputs)

# Apply dropout to LSTM outputs to prevent overfitting
lstm_outs = keras.layers.Dropout(0.2)(lstm_outs)

# Attention Mechanism - Generate attention vectors
input_dim = int(lstm_outs.shape[2])
permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
attention_vector = keras.layers.Reshape((max_words,))(attention_vector)
attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])

# Last layer: fully connected with softmax activation
fc = keras.layers.Dense(embedding_dim, activation='relu')(attention_output)
output = keras.layers.Dense(len(label2id), activation='softmax')(fc)

# Finally building model
model = keras.Model(inputs=[sequence_input], outputs=output)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

# Print model summary
model.summary()





__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 196)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 196, 200)     8402800     input_7[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 196, 200)     0           embedding_7[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 196, 200)     240800      dropout_3[0][0]                  
__________________________________________________________________________________________________
dropout_4 

## Training the model

In [22]:
# Train model 10 iterations
model.fit(X, Y, epochs=2, batch_size=64, validation_split=0.1, shuffle=True)

Train on 1800 samples, validate on 200 samples
Epoch 1/2
1800/1800 [==============================] - 18s 10ms/step - loss: 0.8834 - acc: 0.6761 - val_loss: 0.8265 - val_acc: 0.6950
Epoch 2/2
1800/1800 [==============================] - 16s 9ms/step - loss: 0.8213 - acc: 0.6778 - val_loss: 0.6513 - val_acc: 0.7150


The accuracy on validation data about 93%. Very good result for a classification task with six-classes.
The performance can be further improved by training the model a few more iteration.

**Let's look closer to model predictions and attentions**

In [ ]:
# Re-create the model to get attention vectors as well as label prediction
model_with_attentions = keras.Model(inputs=model.input,
                                    outputs=[model.output, 
                                             model.get_layer('attention_vec').output])

In [ ]:
import random
import math

# Select random samples to illustrate
sample_text = random.choice(dataset["text"].values.tolist())

# Encode samples
tokenized_sample = sample_text.split(" ")
encoded_samples = [[word2id[word] for word in tokenized_sample]]

# Padding
encoded_samples = keras.preprocessing.sequence.pad_sequences(encoded_samples, maxlen=max_words)

# Make predictions
label_probs, attentions = model_with_attentions.predict(encoded_samples)
label_probs = {id2label[_id]: prob for (label, _id), prob in zip(label2id.items(),label_probs[0])}

# Get word attentions using attenion vector
token_attention_dic = {}
max_score = 0.0
min_score = 0.0
for token, attention_score in zip(tokenized_sample, attentions[0][-len(tokenized_sample):]):
    token_attention_dic[token] = math.sqrt(attention_score)


# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
    
def attention2color(attention_score):
    r = 255 - int(attention_score * 255)
    color = rgb_to_hex((255, r, r))
    return str(color)
    
# Build HTML String to viualize attentions
html_text = "<hr><p style='font-size: large'><b>Text:  </b>"
for token, attention in token_attention_dic.items():
    html_text += "<span style='background-color:{};'>{} <span> ".format(attention2color(attention),
                                                                        token)
html_text += "</p>"
# Display text enriched with attention scores 
display(HTML(html_text))

# PLOT EMOTION SCORES
emotions = [label for label, _ in label_probs.items()]
scores = [score for _, score in label_probs.items()]
plt.figure(figsize=(5,2))
plt.bar(np.arange(len(emotions)), scores, align='center', alpha=0.5, color=['black', 'red', 'green', 'blue', 'cyan', "purple"])
plt.xticks(np.arange(len(emotions)), emotions)
plt.ylabel('Scores')
plt.show()



**We have used an attention mechanism with an LSTM network to recognize emotions in given text.
We show that attention mechanism can be useful for classification tasks as well as sequence labeling tasks.
We have illustrated the attentions in order to make model predictions interpretable and look fancy.
Enjoy attentions mechanism in different applications...**

*All feedbacks are welcome.*

